In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras
from matplotlib import pyplot
import imblearn
import gensim
import nltk
from pathlib import Path
#import torch
#import xgboost as xgb
import pickle

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# Load data
df_meta = pd.read_pickle('metadata.pkl')
clicks = pd.read_pickle('clicks.pkl')
adgroup_keywords = pd.read_pickle('adgroup_keywords.pkl')

In [ ]:
#load EMLo arrays
import pickle 

pickle_in = open("elmo_queries_array1.pickle", "rb")
elmo_queries_array1 = pickle.load(pickle_in)

pickle_in = open("elmo_queries_array2.pickle", "rb")
elmo_queries_array2 = pickle.load(pickle_in)


In [ ]:
queries1 = np.concatenate((elmo_queries_array1, elmo_queries_array2))
queries1.shape
pickle_out = open("elmo_queries1.pickle","wb")
pickle.dump(queries1, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("elmo_queries1.pickle", "rb")
elmo_queries = pickle.load(pickle_in)
df_queries = pd.DataFrame(elmo_queries)

In [ ]:
# merge and drop average cost, which is endogenous with target variable

df_features = df_meta.join(df_queries.set_index(df_meta.index))
df_features = df_features.drop(columns=['avg_cost'], axis=1)
df_features.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_queries, clicks,test_size=0.2, random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
#padded_sequences

In [ ]:
#decision tree classifier
#from sklearn.tree import DecisionTreeClassifier 
#from sklearn.metrics import confusion_matrix, accuracy_score

#dtree_model = DecisionTreeClassifier(max_depth = 2).fit(x_train, y_train) 
#dtree_predictions = dtree_model.predict(x_test) 
  
# creating a confusion matrix 
#cm = confusion_matrix(y_test, dtree_predictions) 
#print('accuracy %s' % accuracy_score(dtree_predictions, y_test))

In [ ]:
#Random forest classifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

#Create a Gaussian Classifier
RF_clf=RandomForestClassifier(n_estimators=100)

RF_clf.fit(x_train,y_train)
y_pred=RF_clf.predict(x_test)

print('Accuracy: ',accuracy_score(y_test, y_pred))
print('Precision score: ', precision_score(y_test, y_pred))
print('Recall score: ', recall_score(y_test, y_pred))
print('f1 score: ', f1_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import f1_score

print('f1 score: ', f1_score(y_test, y_pred))

In [ ]:
#random forest feature importance

features = x_train.columns
importances = RF_clf.feature_importances_
indices = np.argsort(importances)[-9:]  # top 10 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
pyplot.title('Feature Importances')
pyplot.barh(range(len(indices)), importances[indices], color='b', align='center')
pyplot.yticks(range(len(indices)), [features[i] for i in indices])
pyplot.xlabel('Relative Importance')
pyplot.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

feature = SelectFromModel(RF_clf)
Fit = feature.fit_transform(x_train, y_train)

In [ ]:
y_pred1=RF_clf.predict(x_test)

print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,y_pred1))
print('Precision score: ', precision_score(y_test, y_pred1))
print('Recall score: ', recall_score(y_test, y_pred1))
print('f1 score: ', f1_score(y_test, y_pred1))

In [ ]:
#param_grid = { 
#    'n_estimators': [200, 500],
#    'max_features': ['auto', 'sqrt', 'log2'],
#    'max_depth' : [4,6,8],
#    'criterion' :['gini', 'entropy']
#}

In [ ]:
#from sklearn.model_selection import GridSearchCV

#CV_rfc = GridSearchCV(estimator=RF_clf, param_grid=param_grid, cv= 3, n_jobs=-1)
#CV_rfc.fit(x_train, y_train)

In [ ]:
#CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')

In [ ]:
rfc1.fit(x_train, y_train)

In [ ]:
pred=rfc1.predict(x_test)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))
print('Precision score: ', precision_score(y_test, pred))
print('Recall score: ', recall_score(y_test, pred))
print('f1 score: ', f1_score(y_test, pred))

In [ ]:
adgroup_keywords = pd.read_pickle('ad_group_keyword_combos.pkl')

In [ ]:
pred_x_test = rfc1.predict_proba(x_test)

In [ ]:
import pickle

rfc_pkl_filename = 'rfc_model.pkl'
# Open the file to save as pkl file
rfc_model_pkl = open(rfc_pkl_filename, 'wb')
pickle.dump(rfc1, rfc_model_pkl)
# Close the pickle instances
rfc_model_pkl.close()

In [3]:
import pickle

rfc_model_pkl = open('rfc_model.pkl', 'rb')
rfc_model = pickle.load(rfc_model_pkl)
print("Loaded model :: ", rfc_model)

Loaded model ::  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [14]:
pickle_in = open("elmo_adgroups_array.pickle", "rb")
elmo_adgroups_array = pickle.load(pickle_in)

adgroup_keywords = pd.read_pickle('adgroup_keywords.pkl')

In [5]:
pred_adgroups = rfc_model.predict_proba(elmo_adgroups_array)[:,1]

In [6]:
pred_adgroups.shape

(386,)

In [13]:
adgroup_keywords['prob_clicks (%)'] = pred_adgroups * 100
adgroup_keywords.sort_values('prob_clicks', ascending=False)

,ad_group,srch_keywords,prob_clicks,prob_clicks (%)
0,3ds Max,mac autodesk version max mac os x autodesk for...,80.236827,80.236827
94,Deep Learning Gpu,gpu deep deep cuda learning nvidia learning cl...,80.214464,80.214464
249,Paul's Keywords,a computer desktop virtual machine service in ...,79.795246,79.795246
360,Visual FX - Houdini,mac houdini engine mac program 15 3d in render...,79.793765,79.793765
190,ML,hosting cloud of server cluster tensor startup...,79.783025,79.783025
276,TensorFlow,gpu platform run on mac windows in good the se...,77.528239,77.528239
265,Revit For Mac,revit autodesk a mac a version download on for...,77.231213,77.231213
268,Run Windows On A Chromebook,convert virtual windows chromebook software on...,77.010176,77.010176
31,Cloud Computer,a gpu computer virtual pc render rendering buy...,77.003495,77.003495
187,Linux Desktop,desktop distros share pc top server the linux ...,76.863764,76.863764
